In [1]:
import pandas as pd
import torch
import torch.nn.functional as F
import sympy as sp

In [12]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

In [8]:
import pandas as pd

df = pd.read_csv('hundo_equations.csv')

functions = []

for _, row in df.iterrows():
    formula = row['Formula']
    num_vars = row['# parameters']
    
    function_details = {
        'formula': formula,
        'num_variables': num_vars,
        'parameters': []
    }
    
    for i in range(1, num_vars + 1):
        param_name = row.get(f'p{i}_name', None)
        if pd.notna(param_name):
            function_details['parameters'].append({
                'name': param_name
            })
    
    functions.append(function_details)

In [11]:
functions[0]

{'formula': 'ax^2 + bx + c',
 'num_variables': 3,
 'parameters': [{'name': 'a'}, {'name': 'b'}, {'name': 'c'}]}

In [30]:
high = 1
low = -1
sequence_size = 110
num_funcs = 10
max_vars = max([f['num_variables'] for f in functions])
sample_size = num_funcs*100

x = torch.linspace(high, low, sequence_size)
# x = x.unsqueeze(1).expand(sequence_size, int(sample_size/num_funcs))

func_data = []
for func in functions:
    hold = []
    for param in func['parameters']:
        p = (high - low) * torch.rand(sample_size) + low
        hold.append(p)
    params = torch.stack(hold)
    params = F.pad(params, (0, 0,0,max_vars - params.shape[0]))
    func_data.append(params)
func_data = torch.stack(func_data)

print(x.shape)
print(func_data.shape)

torch.Size([110])
torch.Size([100, 3, 1000])
